In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './prepped_data/'    
main_data_df = pd.read_csv(basedir + 'main_data_ver_2.4.csv')
print(main_data_df.shape)
main_data_df.head()

(4559866, 16)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,55000,0,817.479191,5,2013,2,1987,26,10.915088,6.706225
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,58250,0,865.784780,4,2013,2,1987,26,10.972499,6.763636
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,60000,0,891.795482,5,2013,5,1987,26,11.002100,6.793237


In [3]:
main_data_df['건물나이'].describe()

count    4.559866e+06
mean     1.391031e+01
std      1.007545e+01
min     -1.200000e+01
25%      6.000000e+00
50%      1.300000e+01
75%      2.100000e+01
max      8.900000e+01
Name: 건물나이, dtype: float64

In [4]:
negative_df = main_data_df[main_data_df['건물나이'] < 0]
negative_df.shape

(5346, 16)

In [5]:
negative_df.head(50)

,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
27078,서울특별시 강남구 도곡동 538-3,서울특별시 강남구 도곡로 320-0,일반,1,아파트_매매,106.390,116900,0,1098.787480,1,2013,9,2016,-3,11.669074,7.001963
142932,서울특별시 강북구 미아동 318-5,서울특별시 강북구 도봉로33길 18-0,일반,1,아파트_매매,68.675,16000,0,232.981434,8,2008,6,2009,-1,9.680344,5.450959
142933,서울특별시 강북구 미아동 318-5,서울특별시 강북구 도봉로33길 18-0,일반,1,아파트_매매,68.675,17500,0,254.823444,8,2008,8,2009,-1,9.769956,5.540571
142934,서울특별시 강북구 미아동 318-5,서울특별시 강북구 도봉로33길 18-0,일반,1,아파트_매매,68.675,19000,0,276.665453,9,2008,8,2009,-1,9.852194,5.622809
439323,서울특별시 노원구 중계동 34-5,서울특별시 노원구 중계로8길 20-0,일반,1,아파트_매매,59.840,24700,0,412.767380,12,2013,3,2018,-5,10.114559,6.022884
439324,서울특별시 노원구 중계동 34-5,서울특별시 노원구 중계로8길 20-0,일반,1,아파트_매매,59.840,22800,0,381.016043,15,2013,5,2018,-5,10.034516,5.942841
439325,서울특별시 노원구 중계동 34-5,서울특별시 노원구 중계로8길 20-0,일반,1,아파트_매매,59.840,23000,0,384.358289,18,2013,5,2018,-5,10.043249,5.951575
439326,서울특별시 노원구 중계동 34-5,서울특별시 노원구 중계로8길 20-0,일반,1,아파트_매매,84.980,34000,0,400.094140,5,2013,5,2018,-5,10.434116,5.991700
439327,서울특별시 노원구 중계동 34-5,서울특별시 노원구 중계로8길 20-0,일반,1,아파트_매매,59.840,24500,0,409.425134,13,2013,5,2018,-5,10.106428,6.014754
439328,서울특별시 노원구 중계동 34-5,서울특별시 노원구 중계로8길 20-0,일반,1,아파트_매매,122.690,39750,0,323.987285,3,2013,5,2018,-5,10.590365,5.780704


In [2]:
def read_data():
    basedir = './prepped_data/'
    
    main_data_df = pd.read_csv(basedir + 'main_data_ver_2.4.csv')
    print(1)
    coor_df = pd.read_csv(basedir + 'coor_data_ver_1.csv')
    print(2)
    land_prices_df = pd.read_csv(basedir + 'land_prices_ver_1.csv')
    print(3)
    land_specs_df = pd.read_csv(basedir + 'land_specs_ver_1.csv')
    print(4)
    land_plans_df = pd.read_csv(basedir + 'land_plans_ver_2.csv')
    print(5)
    
    return main_data_df, coor_df, land_prices_df, land_specs_df, land_plans_df

In [3]:
%%time
main_data_df, coor_df, land_prices_df, land_specs_df, land_plans_df = read_data()

1


c:\users\max6296\ml\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


2
3
4
5
Wall time: 18.4 s
